In [0]:
%pip install pydeequ

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import os
os.environ["SPARK_VERSION"] = "3.5"

print("Reading raw data")
df = spark.read.format("delta").load("dbfs:/FileStore/shared_uploads/traininguser4@sudosu.ai/raw_data")

from pydeequ.checks import Check, CheckLevel
from pydeequ.verification import VerificationSuite,VerificationResult
import json

check = Check(spark, CheckLevel.Error, "Sensor Data Quality")\
    .isContainedIn("Status", ["OK", "WARNING", "ALERT"])

result = VerificationSuite(spark).onData(df).addCheck(check).run()


if result.status != "Success":
    print("Dq checks failed.")
else:
    print("Dq checks passed")

print("Dq results:-")
result_df = VerificationResult.checkResultsAsDataFrame(spark, result)
display(result_df)

output_path = f"dbfs:/FileStore/shared_uploads/traininguser4@sudosu.ai/dq_results"
result_df.write.mode("overwrite").json(output_path)
results_json = [json.loads(row.value) for row in spark.read.text(output_path).collect()]
print(json.dumps(results_json, indent=2))

print("DQ Check complete.")


Reading raw data
Dq checks passed
Dq results:-


/databricks/spark/python/pyspark/sql/dataframe.py:163: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


check,check_level,check_status,constraint,constraint_status,constraint_message
Sensor Data Quality,Error,Success,"ComplianceConstraint(Compliance(Status contained in OK,WARNING,ALERT,`Status` IS NULL OR `Status` IN ('OK','WARNING','ALERT'),None))",Success,


[
  {
    "check": "Sensor Data Quality",
    "check_level": "Error",
    "check_status": "Success",
    "constraint": "ComplianceConstraint(Compliance(Status contained in OK,WARNING,ALERT,`Status` IS NULL OR `Status` IN ('OK','WARNING','ALERT'),None))",
    "constraint_status": "Success",
    "constraint_message": ""
  }
]
DQ Check complete.
